In [17]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def get_image_matrix(image):
    return np.reshape(image,[28,28])

def flip_image_lr(image):
    
#     plt.close('all')
#     f, (ax1, ax2) = plt.subplots(1,2)
#     ax1.imshow(image)
#     ax2.imshow(image_flipped)
#     plt.show()
    
    #reshape the image to 28 X 28
    image = get_image_matrix(image)
    
    #flip the image
    image = np.fliplr(image)
    return image

def add_brightness(image): 
    image = get_image_matrix(image)
    
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
print type(mnist)
batch_size = 100
num_iterations = 1000
summaries_dir = '/home/aayush/ColumbiaCourseProjects/Deep Learning/Summary/'

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
<class 'tensorflow.contrib.learn.python.learn.datasets.base.Datasets'>


In [16]:
#reset the default graph to remove old summary node
tf.reset_default_graph()

#number of samples being fed forward might vary, so None
x = tf.placeholder(tf.float32, [None, 784])
y_actual = tf.placeholder(tf.float32, [None, 10])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))


#learning rate for the gradient descent
eta = tf.placeholder(dtype=tf.float32)

Z = tf.matmul(x, W) + b

#loss per sample
cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(logits = Z, labels = y_actual)
cross_entropy_loss = tf.reduce_mean(cross_entropy_loss)

correct_classifications = tf.equal(tf.argmax(Z, 1), tf.argmax(y_actual, 1))
accuracy = tf.reduce_mean(tf.cast(correct_classifications, tf.float32))


train_step = tf.train.GradientDescentOptimizer(eta).minimize(cross_entropy_loss)

#summary nodes for the network
tf.summary.scalar('Loss', cross_entropy_loss)
tf.summary.scalar('Training Accuracy', accuracy)
merged = tf.summary.merge_all()

print 'Logging summary in {}'.format(summaries_dir + 'train/')
train_writer = tf.summary.FileWriter(summaries_dir + 'train/')


#begin new session
sess = tf.InteractiveSession()

#initialize the variables as specified
init = tf.global_variables_initializer()
init.run()



#run 1000 iterations of training batches of size `batch_size`
for i in range(num_iterations):
    x_batch, y_batch = mnist.train.next_batch(batch_size)
    summary, _ = sess.run([merged,train_step], feed_dict={x: x_batch, y_actual: y_batch, eta: 0.01})
    
    #write the train summary every iteration
    train_writer.add_summary(summary, i)
    

#running the train model on test data set
x_test = mnist.test.images
y_test = mnist.test.labels

accuracy = sess.run(accuracy, feed_dict={x: x_test, y_actual: y_test})
print accuracy

INFO:tensorflow:Summary name Training Accuracy is illegal; using Training_Accuracy instead.
Logging summary in /home/aayush/ColumbiaCourseProjects/Deep Learning/Summary/train/
0.8711
